# 📊 DataMart Analítico - Resumen de Consultas ETL

# Instalación de librerías necesarias (solo ejecutar una vez)

In [1]:
# ⚠️ Instalación solo manual desde Jupyter si hace falta
# !pip install sqlalchemy
# !pip install psycopg2-binary
# !pip install pandas
# !pip install pyyaml

import pandas as pd
import yaml
from sqlalchemy import create_engine

# ------------------ Configuración ------------------

In [2]:
with open('../../configBD/config.yml', 'r') as f:
    cfg = yaml.safe_load(f)
    cfg_etl = cfg['bodega']
    cfg_bd = cfg['mensajeria']

# Crear conexiones a bases de datos

In [3]:
url_etl = f"{cfg_etl['driver']}://{cfg_etl['user']}:{cfg_etl['password']}@{cfg_etl['host']}:{cfg_etl['port']}/{cfg_etl['db']}"
cliente_etl = create_engine(url_etl)

# ------------------ Función auxiliar ------------------

In [4]:
def guardar_datamart(nombre_tabla, query, engine):
    print(f"Creando DataMart: {nombre_tabla}")
    df = pd.read_sql_query(query, engine)
    df.to_sql(nombre_tabla, engine, index=False, if_exists='replace')
    print(f"✔️ {nombre_tabla} guardado correctamente ({len(df)} filas).")


Este notebook genera tablas agregadas (DataMarts) a partir de las consultas más frecuentes extraídas del proceso ETL.
Estas tablas permiten análisis más rápidos al evitar cálculos repetidos sobre las tablas de hechos principales (`fact_servicios` y `fact_novedades`).

**Fecha de generación:** 2025-07-04 04:33:07


### 🧮 resumen_servicios_por_mes

In [5]:
query = """
SELECT "Año", "Mes", COUNT(*) AS total_servicios
FROM fact_servicios
WHERE "EsFinal" = TRUE
GROUP BY "Año", "Mes"
ORDER BY total_servicios DESC
"""
guardar_datamart("resumen_servicios_por_mes", query, cliente_etl)


Creando DataMart: resumen_servicios_por_mes


✔️ resumen_servicios_por_mes guardado correctamente (12 filas).


### 🧮 resumen_servicios_por_dia_semana

In [6]:
query = """
SELECT "DiaSemana", COUNT(*) AS total_servicios
FROM fact_servicios
WHERE "EsFinal" = TRUE
GROUP BY "DiaSemana"
ORDER BY total_servicios DESC
"""
guardar_datamart("resumen_servicios_por_dia_semana", query, cliente_etl)


Creando DataMart: resumen_servicios_por_dia_semana


✔️ resumen_servicios_por_dia_semana guardado correctamente (7 filas).


### 🧮 resumen_actividad_por_hora

In [7]:
query = """
SELECT "Hora", COUNT(*) AS total_servicios
FROM fact_servicios
WHERE "EsFinal" = TRUE
GROUP BY "Hora"
ORDER BY total_servicios DESC
"""
guardar_datamart("resumen_actividad_por_hora", query, cliente_etl)


Creando DataMart: resumen_actividad_por_hora


✔️ resumen_actividad_por_hora guardado correctamente (24 filas).


### 🧮 servicios_por_cliente_mes

In [8]:
query = """
SELECT 
    fs."ClienteKey",
    dc."nombre",
    fs."Año",
    fs."Mes",
    COUNT(*) AS total_servicios
FROM fact_servicios fs
INNER JOIN dim_cliente dc
    ON fs."ClienteKey" = dc."ClienteKey"
WHERE fs."EsFinal" = TRUE
GROUP BY fs."ClienteKey", dc."nombre", fs."Año", fs."Mes"
ORDER BY total_servicios DESC
"""
guardar_datamart("servicios_por_cliente_mes", query, cliente_etl)



Creando DataMart: servicios_por_cliente_mes


✔️ servicios_por_cliente_mes guardado correctamente (102 filas).


### 🧮 eficiencia_mensajeros

In [9]:
query = """
WITH ranked_mensajeros AS (
  SELECT 
    "MensajeroKey",
    COUNT(*) AS servicios_realizados,
    AVG("Duracion_Total_Min") AS duracion_promedio_min,
    AVG("Eficiencia_Mensajero") AS eficiencia_promedio,
    ROW_NUMBER() OVER (
      PARTITION BY AVG("Duracion_Total_Min")
      ORDER BY AVG("Eficiencia_Mensajero") ASC
    ) AS rn
  FROM fact_servicios
  WHERE "EsFinal" = TRUE
  GROUP BY "MensajeroKey"
)
SELECT 
  "MensajeroKey",
  servicios_realizados,
  duracion_promedio_min,
  eficiencia_promedio
FROM ranked_mensajeros
WHERE rn = 1
ORDER BY servicios_realizados DESC, eficiencia_promedio DESC;
"""
guardar_datamart("eficiencia_mensajeros", query, cliente_etl)


Creando DataMart: eficiencia_mensajeros


✔️ eficiencia_mensajeros guardado correctamente (45 filas).


### 🧮 actividad_por_sede_cliente

In [10]:
query = """
	SELECT 
    fs."ClienteKey",
    dc."nombre" AS nombre_cliente,
    ds."nombre" AS nombre_sede_origen,
    COUNT(*) AS total_servicios
FROM fact_servicios fs
INNER JOIN dim_cliente dc
    ON fs."ClienteKey" = dc."ClienteKey"
INNER JOIN dim_sede ds
    ON fs."SedeOrigenKey" = ds."SedeKey"
WHERE "EsFinal" = TRUE
GROUP BY fs."ClienteKey", dc."nombre", fs."SedeOrigenKey", ds."nombre"
ORDER BY total_servicios DESC;
"""
guardar_datamart("actividad_por_sede_cliente", query, cliente_etl)


Creando DataMart: actividad_por_sede_cliente


✔️ actividad_por_sede_cliente guardado correctamente (55 filas).


### 🧮 duracion_promedio_servicio

In [11]:
query = """
SELECT 
    -- Promedio total en minutos (con 2 decimales)
    ROUND(
        CAST(
            AVG("Duracion_Asignacion_Min") + 
            AVG("Duracion_Recogida_Min") + 
            AVG("Duracion_Entrega_Min") + 
            AVG("Duracion_Cierre_Min")
        AS NUMERIC), 
    2) AS tiempo_promedio_min,

    -- Parte entera en horas
    FLOOR((
        AVG("Duracion_Asignacion_Min") + 
        AVG("Duracion_Recogida_Min") + 
        AVG("Duracion_Entrega_Min") + 
        AVG("Duracion_Cierre_Min")
    ) / 60) AS horas,

    -- Parte restante en minutos (módulo 60)
    ROUND(
        MOD(
            CAST(
                AVG("Duracion_Asignacion_Min") + 
                AVG("Duracion_Recogida_Min") + 
                AVG("Duracion_Entrega_Min") + 
                AVG("Duracion_Cierre_Min")
            AS NUMERIC), 
        60)
    ) AS minutos,

    -- Formato como texto: "X horas y Y minutos"
    CONCAT(
        FLOOR((
            AVG("Duracion_Asignacion_Min") + 
            AVG("Duracion_Recogida_Min") + 
            AVG("Duracion_Entrega_Min") + 
            AVG("Duracion_Cierre_Min")
        ) / 60), ' horas y ',
        ROUND(
            MOD(
                CAST(
                    AVG("Duracion_Asignacion_Min") + 
                    AVG("Duracion_Recogida_Min") + 
                    AVG("Duracion_Entrega_Min") + 
                    AVG("Duracion_Cierre_Min")
                AS NUMERIC), 
            60)
        ), ' minutos'
    ) AS tiempo_promedio_formateado

FROM fact_servicios
WHERE "EsFinal" = TRUE;
"""
guardar_datamart("duracion_promedio_servicio", query, cliente_etl)

Creando DataMart: duracion_promedio_servicio


✔️ duracion_promedio_servicio guardado correctamente (1 filas).


### 🧮 duracion_promedio_por_fase

In [12]:
query = """
SELECT 
    AVG("Duracion_Asignacion_Min") AS asignacion_promedio_min,
    AVG("Duracion_Recogida_Min") AS recogida_promedio_min,
    AVG("Duracion_Entrega_Min") AS entrega_promedio_min,
    AVG("Duracion_Cierre_Min") AS cierre_promedio_min
FROM fact_servicios
WHERE "EsFinal" = TRUE;
"""
guardar_datamart("duracion_promedio_por_fase", query, cliente_etl)

Creando DataMart: duracion_promedio_por_fase


✔️ duracion_promedio_por_fase guardado correctamente (1 filas).


### 🧮 novedades_por_tipo

In [13]:
query = """
SELECT 
    "TipoNovedad" AS "Tipo de Novedad",
    "Gravedad" AS "Nivel de Gravedad",
    COUNT(*) AS "Total de Ocurrencias"
FROM fact_novedades
GROUP BY "TipoNovedad", "Gravedad"
ORDER BY "Total de Ocurrencias" DESC;
"""
guardar_datamart("novedades_por_tipo", query, cliente_etl)

Creando DataMart: novedades_por_tipo
✔️ novedades_por_tipo guardado correctamente (2 filas).
